<a href="https://colab.research.google.com/github/ryandoyle5401/OS_Project1/blob/main/proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%writefile proj1.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/time.h>
#include <sys/resource.h>

#define MAX_WORDS 50
#define MAX_WORD_LENGTH 20
#define NUM_OF_PROCESSES 7

int main() {
    // Store paths to all 7 files
    char *file_paths[] = {
        "/content/drive/MyDrive/Colab Notebooks/txtfiles/bib",
        "/content/drive/MyDrive/Colab Notebooks/txtfiles/paper1",
        "/content/drive/MyDrive/Colab Notebooks/txtfiles/paper2",
        "/content/drive/MyDrive/Colab Notebooks/txtfiles/progc",
        "/content/drive/MyDrive/Colab Notebooks/txtfiles/progl",
        "/content/drive/MyDrive/Colab Notebooks/txtfiles/progp",
        "/content/drive/MyDrive/Colab Notebooks/txtfiles/trans"
    };

    // Create pipes for each process
    int pipes[NUM_OF_PROCESSES][2];
    for (int i = 0; i < NUM_OF_PROCESSES; i++) {
        if (pipe(pipes[i]) == -1) {
            perror("failed to create pipe");
            exit(1);
        }
    }

    // Keywords to keep track of
    char keywords[MAX_WORDS][MAX_WORD_LENGTH] = {
        "and", "as", "at", "be", "but", "by", "for", "if", "in", "into",
        "is", "it", "no", "not", "of", "on", "or", "such", "that", "the",
        "their", "then", "there", "these", "they", "this", "to", "was",
        "will", "with", "a", "an", "are", "can", "could", "have", "me",
        "my", "you", "your", "we", "were", "good", "do", "from", "what",
        "so", "like", "just", "some"
    };

    // Array to hold frequencies for each keyword
    int frequencies[MAX_WORDS] = {0};

    // Start time measurement
    struct timeval start, end;
    gettimeofday(&start, NULL);  // Record the start time

    pid_t pid[NUM_OF_PROCESSES];
    for (int i = 0; i < NUM_OF_PROCESSES; i++) {
        pid[i] = fork();
        if (pid[i] == -1) {
            perror("failed to create child process");
            exit(1);
        } else if (pid[i] == 0) {
            // Child process
            FILE *file_ptr = fopen(file_paths[i], "r");
            if (file_ptr == NULL) {
                perror("failed to open file");
                exit(1);
            }

            int local_frequencies[MAX_WORDS] = {0}; // Local frequencies for this process
            char str[256];

            while (fgets(str, sizeof(str), file_ptr) != NULL) {
                // Tokenize the string
                char *token = strtok(str, " \n\t.,;:!?");
                while (token != NULL) {
                    // Check each keyword against the token
                    for (int j = 0; j < MAX_WORDS; j++) {
                        if (strcmp(token, keywords[j]) == 0) {
                            local_frequencies[j]++;
                        }
                    }
                    token = strtok(NULL, " \n\t.,;:!?");
                }
            }
            fclose(file_ptr);

            // Send the frequencies to the parent process via the pipe
            write(pipes[i][1], local_frequencies, sizeof(local_frequencies));
            close(pipes[i][1]);  // Close the write end after sending
            exit(0);  // Child process exits after completion
        }
    }

    // Parent process: collect counts from children
    for (int i = 0; i < NUM_OF_PROCESSES; i++) {
        close(pipes[i][1]);  // Close the write end of the pipe in the parent

        int child_frequencies[MAX_WORDS];
        read(pipes[i][0], child_frequencies, sizeof(child_frequencies));  // Read the frequencies from the child

        // Accumulate frequencies in the parent
        for (int j = 0; j < MAX_WORDS; j++) {
            frequencies[j] += child_frequencies[j];
        }

        close(pipes[i][0]);  // Close the read end after reading
    }

    // Wait for all child processes to finish
    int status;
    pid_t wpid;
    for (int i = 0; i < NUM_OF_PROCESSES; i++) {
        wpid = wait(&status);
    }

    // End time measurement
    gettimeofday(&end, NULL);  // Record the end time

    // Calculate elapsed time in seconds and microseconds
    long seconds = end.tv_sec - start.tv_sec;
    long microseconds = end.tv_usec - start.tv_usec;
    long total_time = (seconds * 1000000) + microseconds;

    // Get resource usage
    struct rusage usage;
    getrusage(RUSAGE_CHILDREN, &usage); // Get resource usage for child processes

    // Display frequencies of keywords
    printf("Frequencies of keywords:\n");
    for (int j = 0; j < MAX_WORDS; j++) {
        if (frequencies[j] > 0) {  // Only print if the frequency is greater than zero
            printf("'%s': %d\n", keywords[j], frequencies[j]);
        }
    }

    printf("Total execution time: %ld microseconds.\n", total_time);
    printf("User CPU time used: %ld microseconds.\n", usage.ru_utime.tv_sec * 1000000 + usage.ru_utime.tv_usec);
    printf("System CPU time used: %ld microseconds.\n", usage.ru_stime.tv_sec * 1000000 + usage.ru_stime.tv_usec);
    printf("Maximum resident set size: %ld kilobytes.\n", usage.ru_maxrss);

    return EXIT_SUCCESS;
}


Overwriting proj1.c


In [24]:
%%shell
gcc proj1.c -o proj1
./proj1

Frequencies of keywords:
'and': 862
'as': 189
'at': 84
'be': 325
'but': 58
'by': 188
'for': 465
'if': 415
'in': 650
'into': 57
'is': 528
'it': 356
'no': 65
'not': 120
'of': 1192
'on': 263
'or': 108
'such': 30
'that': 282
'the': 1568
'their': 45
'then': 245
'there': 34
'these': 24
'they': 56
'this': 183
'to': 783
'was': 65
'will': 131
'with': 217
'a': 783
'an': 158
'are': 150
'can': 134
'could': 63
'have': 80
'me': 16
'my': 38
'you': 100
'your': 39
'we': 58
'were': 16
'good': 12
'do': 140
'from': 116
'what': 29
'so': 59
'like': 43
'just': 44
'some': 55
Total execution time: 68453 microseconds.
User CPU time used: 110566 microseconds.
System CPU time used: 35060 microseconds.
Maximum resident set size: 20648 kilobytes.
